In [1]:
import gzip
import pandas as pd
import numpy as np
from datetime import datetime
import re
from collections import Counter
import bs4
import requests

# Data Cleaning

### User Reviews

In [2]:
def strip_funny(input):
    if input == 0:
        return 0
    return int(input.split()[0].replace(',',''))

In [3]:
month_mapping = {'January':'1', 'February':'2', 'March':'3', 'April':'4', 'May':'5', 'June':'6', 'July':'7', 'August':'8',
                 'September':'9', 'October':'10', 'November':'11', 'December':'12'}

In [4]:
def parse_posted(input):
    
    # range-probability for choosing random filler year
    year_range = [2014, 2015, 2013, 2012, 2011, 2010]
    year_prob = [0.44953849, 0.37551336, 0.13808807, 0.02470215, 0.01081609, 0.00134185]
    
    temp = input[7:-1].replace(',','')
    for month in month_mapping.keys():
        temp = temp.replace(month, month_mapping[month])
            
    if re.match(r'^\d{1,2} \d{1,2}$',temp):
        month, day = temp.split()
        month = month.zfill(2)
        day = day.zfill(2)
        if (month=='02') and (day=='29'):
            day = '28'
        rand_year = np.random.choice(a=year_range, p=year_prob)
        return str(rand_year) + '-' + month + '-' + day
    else:
        month, day, year = temp.split()
        month = month.zfill(2)
        day = day.zfill(2)
        if (month=='02') and (day=='29'):
            day = '28'
        return year + '-' + month + '-' + day

In [5]:
def parse_helpful(input):
    # FILLS NA HELPFUL RATINGS TO 0
    if input == 'No ratings yet':
        return 0
    else:
        return int(input[input.index('(')+1:input.index(')')-1])/100

In [6]:
user_reviews = pd.read_csv('data\\user_reviews.csv')

In [7]:
user_reviews['recommend'].value_counts()

True     52473
False     6832
Name: recommend, dtype: int64

In [8]:
user_reviews = pd.read_csv('data\\user_reviews.csv')
user_reviews['funny'] = user_reviews['funny'].fillna(value=0).apply(strip_funny)
user_reviews['posted'] = pd.to_datetime(user_reviews['posted'].apply(parse_posted), format="%Y-%m-%d").apply(lambda x: x.date())
user_reviews['posted'] = pd.to_datetime(user_reviews['posted'])
user_reviews['helpful'] = user_reviews['helpful'].apply(parse_helpful)
user_reviews['review'] = user_reviews['review'].fillna('')
user_reviews = user_reviews.drop(columns=['last_edited'])
user_reviews.head()

,user_id,user_url,funny,posted,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,2011-11-05,1250,0.00,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,2011-07-15,22200,0.00,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,2011-04-21,43110,0.00,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,0,2014-06-24,251610,0.75,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,0,2013-09-08,227300,0.00,True,For a simple (it's actually not all that simpl...


In [9]:
for col in user_reviews.columns:
    print(col, sum(user_reviews[col].isna()))

user_id 0
user_url 0
funny 0
posted 0
item_id 0
helpful 0
recommend 0
review 0


In [10]:
user_reviews.dtypes

user_id              object
user_url             object
funny                 int64
posted       datetime64[ns]
item_id               int64
helpful             float64
recommend              bool
review               object
dtype: object

### User Games

In [11]:
user_games = pd.read_csv('data\\user_games.csv')
user_games.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0


In [12]:
for col in user_games.columns:
    print(col, sum(user_games[col].isna()))

user_id 0
items_count 0
steam_id 0
user_url 0
item_id 0
item_name 0
playtime_forever 0
playtime_2weeks 0


In [13]:
user_games.dtypes

user_id             object
items_count          int64
steam_id             int64
user_url            object
item_id              int64
item_name           object
playtime_forever     int64
playtime_2weeks      int64
dtype: object

### Steam Games

In [14]:
games_data = []
f = gzip.open("data\\steam_games.json.gz")
for l in f:
    d = eval(l)
    games_data.append(d) 
f.close()
games = pd.DataFrame(games_data)
games.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN


In [15]:
games

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.49,http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",NaN,NaN
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",4.24,http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,NaN,NaN
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",1.39,http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,NaN,NaN
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",NaN,http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns",1 user reviews,NaN


In [16]:
genre_list = []
for index, input in games['genres'].items():
    if type(input) == list:
        genre_list += input
genre_counts = Counter(genre_list)

In [17]:
def parse_genre(input, genre):
    if type(input) == list:
        return input.count(genre)
    else:
        return 0

In [18]:
tags_list = []
for index, input in games['tags'].items():
    if type(input) == list:
        tags_list += input
tags_counts = Counter(tags_list)

In [19]:
def parse_tag(input, tag):
    if type(input) == list:
        return input.count(tag)
    else:
        return 0

In [20]:
specs_list = []
for index, input in games['specs'].items():
    if type(input) == list:
        specs_list += input
specs_counts = Counter(specs_list)

In [21]:
def parse_spec(input, spec):
    if type(input) == list:
        return input.count(spec)
    else:
        return 0

#### Select Relevant Columns

In [22]:
games = games[['developer', 'app_name', 'url', 'genres', 'tags', 'specs','release_date', 'price', 'id']]
games['developer'] = games['developer'].fillna('')
games = games.dropna(subset=['app_name'])
games = games.dropna(subset=['id'])
games.head()

C:\Users\Innominandum\AppData\Local\Temp\ipykernel_22656\2145617593.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['developer'] = games['developer'].fillna('')


,developer,app_name,url,genres,tags,specs,release_date,price,id
0,Kotoshiro,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],2018-01-04,4.99,761140
1,Secret Level SRL,Ironbound,http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",2018-01-04,Free To Play,643980
2,Poolians.com,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",2017-07-24,Free to Play,670290
3,彼岸领域,弹炸人2222,http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]","[Action, Adventure, Casual]",[Single-player],2017-12-07,0.99,767400
4,,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",NaN,2.99,773570


#### Normalize Genres, Tags, and Specs

In [23]:
genres_one_hots = []
for genre in genre_counts.keys():
    genres_one_hots += [games['genres'].apply(lambda input: parse_genre(input, genre)).rename(genre)]
genres = pd.concat(genres_one_hots, axis=1)

In [24]:
tags_one_hots = []
for tag in tags_counts.keys():
    tags_one_hots += [games['tags'].apply(lambda input: parse_tag(input, tag)).rename(tag)]
tags = pd.concat(tags_one_hots, axis=1)

In [25]:
specs_one_hots = []
for spec in specs_counts.keys():
    specs_one_hots += [games['specs'].apply(lambda input: parse_spec(input, spec)).rename(spec)]
specs = pd.concat(specs_one_hots, axis=1)

In [26]:
with_dupe_indentifiers = pd.concat([genres, tags, specs], axis=1)

In [27]:
identifier_list = []
for identifier in set(with_dupe_indentifiers.columns):
    if len(with_dupe_indentifiers[identifier].shape) == 1:
        identifier_list += [with_dupe_indentifiers[identifier]]
    else:
        temp = with_dupe_indentifiers[identifier].iloc(axis=1)[0]
        for i in np.arange(1, with_dupe_indentifiers[identifier].shape[1]):
            temp = temp + with_dupe_indentifiers[identifier].iloc[:,i]
            temp = temp.apply(lambda x: 0 if x == 0 else 1 )
        identifier_list += [temp]

In [28]:
identifier_list = [games.drop(columns=['genres', 'tags', 'specs'])] + identifier_list
games_with_identifiers = pd.concat(identifier_list, axis=1)
games_with_identifiers.head()

,developer,app_name,url,release_date,price,id,Otome,Board Game,Management,Online Co-op,...,War,Medieval,Fantasy,Level Editor,Steam Achievements,Massively Multiplayer,World War I,Turn-Based,Replay Value,Tactical
0,Kotoshiro,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,4.99,761140,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Secret Level SRL,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,Free To Play,643980,0,1,0,0,...,0,0,1,0,1,0,0,1,1,1
2,Poolians.com,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,Free to Play,670290,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,彼岸领域,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,0.99,767400,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,2.99,773570,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Imputate Price - Webscraper

In [29]:
def scrape_price(url):
    #url = 'https://store.steampowered.com/app/582010/Monster_Hunter_World/'
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text)
    
    list_of_prices = soup.find_all('div', attrs={'class':'game_purchase_price price'})
    
    # NO ORIGINAL PRICE AVAILABLE
    if len(list_of_prices) == 0:
        
        # DISCOUNTED PRICE
        discount_price = soup.find('div', attrs={'class':'discount_original_price'})
        if discount_price is not None:
            return float(discount_price.text.strip().replace('$',''))
        else:
            # COMING SOON
            if soup.find('h1', text='Coming soon') is not None:
                return 'Coming soon'
            else:
                # FREE
                if soup.find('span', text='Free') is not None:
                    return 'Free'
                # NOT PURCHASABLE
                else:
                    return 'Not purchasable'
    else:
        for i, price in enumerate(list_of_prices):
            # SKIP DEMO PRICES
            if 'demo' in price.text.strip().lower():
                if i == len(list_of_prices) - 1:
                    return 'Free'
                else:
                    continue
            # OPEN BETA AND DISCOUNTED
            elif price.text.strip().lower() == 'Open Beta':
                discount_price = soup.find('div', attrs={'class':'discount_original_price'})
                return float(discount_price.text.strip().replace('$',''))
            else:
                final_price = price.text.strip().replace('$','')
                # FREE
                if 'free' in final_price.lower():
                    return 'Free'
                # ORIGINAL PRICE
                else:
                    # Railgunners
                    if final_price == '730640':
                        return 'Free'
                    try:
                        return float(final_price)
                    # GENERAL CATCH
                    except ValueError:
                        if i == len(list_of_prices) - 1:
                            # Dead In Bermuda
                            if final_price == 'Try to survive the 5 first days on the island!':
                                return 14.99
                            # Enigma Prison Beta
                            elif final_price == 'Enigma Prison Beta':
                                return 'Coming soon'
                            else:
                                return final_price
                        else:
                            continue

In [30]:
%%time
scraped_prices = games_with_identifiers[games_with_identifiers['price'].isna()]['url'].apply(scrape_price)

CPU times: total: 1min 51s
Wall time: 12min 18s


In [31]:
for i in scraped_prices.index:
    games_with_identifiers.loc[i,'price'] = scraped_prices.loc[i]

#### Imputate Release Date - Webscraper

In [32]:
def scrape_release_date(url):
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text)
    temp_date = soup.find('div', attrs={'class':'date'})
    if temp_date is None:
        return 'No Release Date'
    return temp_date.text

In [33]:
abbrv_month_mapping = {'Jan':'1', 'Feb':'2', 'Mar':'3', 'Apr':'4', 'May':'5', 'Jun':'6', 'Jul':'7', 'Aug':'8',
                 'Sep':'9', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

In [34]:
def process_scraped_date(date):
    if (date == 'No Release Date') or (date == 'Coming soon'):
        return date
        
    temp_date = date
    for month in abbrv_month_mapping.keys():
        temp_date = temp_date.replace(month, abbrv_month_mapping[month])
        
    date_list = temp_date.replace(',','').split()

    if len(date_list) < 3:
        month, year = date_list
        month = month.zfill(2)
        day = '01' #default date
    else:
        month, day, year = date_list
        month = month.zfill(2)
        day = day.zfill(2)
    return year + '-' + month + '-' + day

In [35]:
%%time
scraped_release_dates = games_with_identifiers[games_with_identifiers['release_date'].isna()]['url'].apply(scrape_release_date)

CPU times: total: 2min 34s
Wall time: 18min 17s


In [36]:
scraped_release_dates = scraped_release_dates.apply(process_scraped_date)

In [37]:
for i in scraped_release_dates.index:
    games_with_identifiers.loc[i,'release_date'] = scraped_release_dates.loc[i]

#### Clean Price and Release Date

In [38]:
def fix_free_prices(price):
    if (type(price) == str) and ('free' in price.lower()):
        return float(0)
    else:
        return price

In [39]:
def abnormal_prices(prices):
    return prices[prices.apply(lambda price: type(price)!=float)].value_counts()

In [40]:
# drop rows with abnormal prices
abnormal_price_vals = list(abnormal_prices(games_with_identifiers['price'].apply(fix_free_prices)).index)
games_with_identifiers = games_with_identifiers[games_with_identifiers['price'].apply(lambda price: price not in abnormal_price_vals)]

In [41]:
# fix free-2-play games
games_with_identifiers['price'] = games_with_identifiers['price'].apply(fix_free_prices)

In [42]:
num_days_month = {1:31, 2:29, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

In [43]:
month_range = [10, 11, 12, 9, 8, 7, 6, 5, 4, 3, 2, 1]
month_prob = [0.10718511, 0.09883925, 0.09855147, 0.0953858 , 0.09161257,0.08256323, 0.07677549, 0.07619992, 0.07428133, 0.07156333,0.06443258, 0.06260992]

jan_range = [ 1, 26, 15, 27, 29, 19, 13, 31, 20,  6, 12,  8, 30, 22,  3, 14, 28, 23,  7, 21, 10, 16, 25, 24,  9,  5,  4,  2, 17, 18, 11]
jan_prob = [0.09908069, 0.04749745, 0.04443309, 0.04392237, 0.04085802, 0.04034729, 0.03881512, 0.03830439, 0.03728294, 0.03575077, 0.03575077, 0.03421859, 0.03370787, 0.03319714, 0.02911134, 0.02808989, 0.02808989, 0.02706844, 0.02604699, 0.02553626, 0.02502554, 0.02451481, 0.02400409, 0.02298264, 0.02196118, 0.02145046, 0.01991828, 0.01940756, 0.01787538, 0.01787538, 0.01787538]
feb_range = [24, 14,  1,  3,  6, 27, 17, 21, 28,  2, 13, 23, 26, 19, 25, 16, 10, 15,  9, 18, 12, 20, 22,  7,  8, 11,  5,  4]
feb_prob = [0.05941589, 0.04632427, 0.04279959, 0.04179255, 0.04128902, 0.0407855 , 0.04028197, 0.03977845, 0.03927493, 0.03927493, 0.03776435, 0.03776435, 0.03726083, 0.0367573 , 0.03625378, 0.03575025, 0.03575025, 0.03474321, 0.03474321, 0.03373615, 0.02920443, 0.02920443, 0.0287009 , 0.02819738, 0.02668681, 0.02517624, 0.02215509, 0.01913394]
mar_range = [28, 10,  1, 17,  8, 15, 30, 31, 25, 24, 20, 21,  7,  3, 14, 29, 23, 27, 16,  4,  9,  2, 22, 18,  6, 26, 11, 13, 12, 19,  5]
mar_prob = [0.05942806, 0.0487042 , 0.04602324, 0.04423592, 0.04200179, 0.04155496, 0.04110813, 0.0406613 , 0.03932082, 0.03842717, 0.03663986, 0.03663986, 0.03663986, 0.03619303, 0.0357462 , 0.03217158, 0.02993744, 0.02859696, 0.02859696, 0.02636282, 0.02636282, 0.02636282, 0.02502234, 0.02368186, 0.02323503, 0.02234138, 0.02055407, 0.01831993, 0.01519214, 0.01519214, 0.01474531]
apr_range = [ 5, 26, 28,  7, 19, 12, 18, 21,  3, 25,  1,  4, 29, 11, 13, 27, 14, 22, 24, 30, 20, 17,  2,  6, 15, 10,  8, 16, 23,  9]
apr_prob = [0.0525183 , 0.04993543, 0.04864399, 0.04649161, 0.04218683, 0.04089539, 0.04046492, 0.04003444, 0.03917348, 0.03702109, 0.03659062, 0.03616014, 0.03529918, 0.03400775, 0.03400775, 0.03314679, 0.03185536, 0.03142488, 0.03142488, 0.03056393, 0.03013345, 0.0271201 , 0.02539819, 0.02496771, 0.02453724, 0.02367628, 0.02281533, 0.01851055, 0.01721911, 0.01377529]
may_range = [26,  1, 12, 19, 11,  5, 30, 29, 31, 15, 18,  4, 22, 16,  9, 20,  6, 25, 27, 10, 17, 23,  2, 24, 28,  8,  3, 21, 13, 14,  7]
may_prob = [0.05329417, 0.0482585 , 0.0461603 , 0.04406211, 0.04154427, 0.04154427, 0.04112463, 0.04112463, 0.03692824, 0.0365086 , 0.03441041, 0.03441041, 0.03357113, 0.03357113, 0.03315149, 0.03231221, 0.03231221, 0.03147293, 0.03063366, 0.03021402, 0.02811582, 0.02727654, 0.02517835, 0.02475871, 0.02433907, 0.02349979, 0.02308015, 0.01888376, 0.01846412, 0.01678556, 0.01300881]
jun_range = [ 1, 21, 16,  8,  2, 20,  9,  6, 30, 15, 10, 22,  3, 13, 23, 26,  7, 19, 12, 17, 27,  5, 24,  4, 25, 28, 14, 29, 11, 18]
jun_prob = [0.05914202, 0.05747605, 0.0545606 , 0.0487297 , 0.0487297 , 0.04748022, 0.04498126, 0.03956685, 0.03915035, 0.03748438, 0.0366514 , 0.03498542, 0.03456893, 0.03415244, 0.03373594, 0.030404  , 0.02915452, 0.02873803, 0.02707205, 0.02498959, 0.02498959, 0.02457309, 0.02457309, 0.0241566 , 0.02332362, 0.02207414, 0.02124115, 0.01874219, 0.01332778, 0.01124531]
jul_range = [14,  7, 25, 31, 21, 26, 11, 20, 28, 13,  5, 12,  1, 18, 24, 29, 10, 17, 22, 27,  6, 19, 15, 23,  8, 16,  3,  4, 30,  2,  9]
jul_prob = [0.04957401, 0.04841208, 0.0464756 , 0.04492641, 0.04453912, 0.04027885, 0.03911696, 0.03834237, 0.03795507, 0.03718048, 0.03563129, 0.035244  , 0.0344694 , 0.03330751, 0.03253292, 0.03214562, 0.03098373, 0.03098373, 0.03020914, 0.03020914, 0.02865995, 0.02788536, 0.02711077, 0.02517428, 0.02439969, 0.02130132, 0.02091402, 0.02013943, 0.01859024, 0.01742835, 0.01587916]
aug_range = [31, 30, 25,  1, 22, 29, 11,  8,  4, 15,  2,  3, 24, 18, 16, 23, 21, 9, 10, 26, 17,  7, 28, 20, 19, 12,  6, 14, 27,  5, 13]
aug_prob = [0.05375218, 0.0495637 , 0.04816754, 0.04712042, 0.04223386, 0.03979058, 0.03874346, 0.03804538, 0.03804538, 0.03804538, 0.03630017, 0.03560209, 0.03525305, 0.03246073, 0.03176265, 0.03141361, 0.03071553, 0.02897033, 0.02792321, 0.02687609, 0.02687609, 0.02687609, 0.02617801, 0.02513089, 0.02408377, 0.02408377, 0.02129145, 0.02129145, 0.02024433, 0.01884817, 0.01431065]
sep_range = [29, 15,  1, 26, 28, 22,  8, 12, 16, 30, 13, 19, 14, 20, 27, 23, 21, 25, 18,  9,  5,  7, 24,  4,  6,  2, 17, 11, 10,  3]
sep_prob = [0.06134764, 0.05631914, 0.05363728, 0.04793832, 0.04022796, 0.03989273, 0.03888703, 0.03855179, 0.0365404 , 0.03620516, 0.03620516, 0.03586993, 0.03586993, 0.03519946, 0.034529, 0.03419376, 0.03385853, 0.03117667, 0.0295005 , 0.02815957, 0.02815957, 0.02681864, 0.02648341, 0.02346631, 0.02313108, 0.02044921, 0.01910828, 0.01810258, 0.01542072, 0.01475025]
oct_range = [16, 20, 25, 26, 31, 27, 17,  3, 13, 24, 28, 30, 19,  6,  5, 23, 12, 14, 10,  4, 21, 18, 29, 11,  7,  1,  9, 15, 22,  2,  8]
oct_prob = [0.05966588, 0.04743437, 0.04564439, 0.04534606, 0.04445107, 0.04236277, 0.04236277, 0.03937947, 0.03848449, 0.03818616, 0.03579952, 0.03460621, 0.03341289, 0.03281623, 0.03162291, 0.03072792, 0.03072792, 0.03042959, 0.03013126, 0.03013126, 0.02774463, 0.02684964, 0.025358  , 0.02446301, 0.02267303, 0.02118138, 0.02088305, 0.01909308, 0.01909308, 0.01819809, 0.01073986]
nov_range = [14,  1, 17, 30, 15,  3,  9,  2, 18, 10, 20, 21,  7, 16,  8, 22, 11, 24, 28, 13,  5, 19,  6, 12, 29,  4, 23, 25, 27, 26]
nov_prob = [0.04626331, 0.04529279, 0.04464575, 0.04432223, 0.04399871, 0.04173407, 0.04108703, 0.04011647, 0.03720479, 0.03688127, 0.03655775, 0.03655775, 0.03591071, 0.03396959, 0.03332255, 0.03299903, 0.03170495, 0.03170495, 0.03138143, 0.03105791, 0.03008735, 0.03008735, 0.02911679, 0.02911679, 0.02879327, 0.02782271, 0.02102879, 0.02070527, 0.01488192, 0.01164672]
dec_range = [15,  1, 16, 20,  7, 19,  8,  5, 21, 14,  9, 18, 13, 12,  6,  4, 22, 11, 10,  2,  3, 17, 29, 23, 27, 28, 31, 25, 30, 26, 24]
dec_prob = [0.06554183, 0.05743024, 0.0486697 , 0.04834523, 0.04834523, 0.04639844, 0.04574951, 0.04477612, 0.0431538 , 0.04120701, 0.03926022, 0.03861129, 0.03698897, 0.03698897, 0.03698897, 0.0366645 , 0.03504218, 0.03439325, 0.03277093, 0.02855289, 0.02498378, 0.02238806, 0.02011681, 0.01784555, 0.01524984, 0.01265412, 0.0110318 , 0.00811162, 0.00746269, 0.00746269, 0.00681376]

month_range_prob = {1: (jan_range, jan_prob), 2: (feb_range, feb_prob), 3: (mar_range, mar_prob), 4: (apr_range, apr_prob), 
                    5: (may_range, may_prob), 6: (jun_range, jun_prob), 7: (jul_range, jul_prob), 8: (aug_range, aug_prob), 
                    9: (sep_range, sep_prob), 10: (oct_range, oct_prob), 11: (nov_range, nov_prob), 12: (dec_range, dec_prob)}

In [44]:
def fix_month_year(date):
    
    date_list = date.replace(',','').split()
    if len(date_list) == 2:
        month, year = date_list
        if year.isdigit():
            if month in month_mapping.keys():
                month = month_mapping[month]
                
                day_range, day_prob = month_range_prob[int(month)]
                
                rand_day = np.random.choice(a=day_range, p=day_prob)
                
                return year + '-' + month + '-' + str(rand_day)
            elif month in abbrv_month_mapping.keys():
                month = abbrv_month_mapping[month]
                
                day_range, day_prob = month_range_prob[int(month)]
                
                rand_day = np.random.choice(a=day_range, p=day_prob)
                
                return year + '-' + month + '-' + str(rand_day)
                
    return date

In [45]:
def fix_year_only(date):
    
    rand_month = np.random.choice(a=month_range, p=month_prob)
    
    day_range, day_prob = month_range_prob[rand_month]
                
    rand_day = np.random.choice(a=day_range, p=day_prob)

    if (date.isdigit()) and (len(date) == 4):
        return date + '-' + str(rand_month) + '-' + str(rand_day)
    return date

In [46]:
def abnormal_date(dates):
    
    def helper(date):
        if type(date) == str:
            if date.replace('-','').isdigit():
                if len(date.split('-')) == 3:
                    return False
            return True
        
    return dates[dates.apply(helper)].value_counts()

In [47]:
# drop rows with abnormal release dates
abnormal_date_vals = list(abnormal_date(games_with_identifiers['release_date'].apply(fix_month_year).apply(fix_year_only)).index)
games_with_identifiers = games_with_identifiers[games_with_identifiers['release_date'].apply(lambda date: date not in abnormal_date_vals)]

In [48]:
# fix games with only month-year or year dates
games_with_identifiers['release_date'] = games_with_identifiers['release_date'].apply(fix_month_year)
games_with_identifiers['release_date'] = games_with_identifiers['release_date'].apply(fix_year_only)

In [49]:
# convert to datetime
games_with_identifiers['release_date'] = pd.to_datetime(games_with_identifiers['release_date'], format="%Y-%m-%d").apply(lambda x: x.date())
games_with_identifiers['release_date'] = pd.to_datetime(games_with_identifiers['release_date'])

In [50]:
# convert id column to int64
games_with_identifiers['id'] = games_with_identifiers['id'].astype('int64')

In [51]:
games_with_identifiers.head()

,developer,app_name,url,release_date,price,id,Otome,Board Game,Management,Online Co-op,...,War,Medieval,Fantasy,Level Editor,Steam Achievements,Massively Multiplayer,World War I,Turn-Based,Replay Value,Tactical
0,Kotoshiro,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,4.99,761140,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Secret Level SRL,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,0.00,643980,0,1,0,0,...,0,0,1,0,1,0,0,1,1,1
2,Poolians.com,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,0.00,670290,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,彼岸领域,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,0.99,767400,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,2018-01-04,2.99,773570,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
for col in games_with_identifiers.columns:
    print(col, sum(games_with_identifiers[col].isna()))
    if col == 'id':
        break

developer 0
app_name 0
url 0
release_date 0
price 0
id 0


In [53]:
games_with_identifiers.dtypes.head(10)

developer               object
app_name                object
url                     object
release_date    datetime64[ns]
price                  float64
id                       int64
Otome                    int64
Board Game               int64
Management               int64
Online Co-op             int64
dtype: object

In [54]:
# user_reviews.to_csv('user_reviews_clean.csv', index=False)
# games_with_identifiers.to_csv('steam_games_clean.csv', index=False)